In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [50]:
import sqlite3

In [51]:
# Load data into a dataframe
conn = sqlite3.connect('original_heart_database.db')
query = "SELECT * FROM cut_heart_disease;"
df = pd.read_sql_query(query, conn)
conn.close()

In [52]:
string_columns = [v for v in df.select_dtypes(exclude=['float64', 'int64']).columns if v != 'target']
df = df.drop(string_columns, axis=1)

Y = df['target']
X = df.drop('target', axis=1)

In [53]:
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

In [54]:
def train_logistic_regression(X_train, y_train, X_val, y_val):
    # Create an instance of Logistic Regression model
    lr_model = LogisticRegression()

    # Train the model
    lr_model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = lr_model.predict(X_val)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_val, y_pred)

    # Perform cross-validation
    cv_scores = cross_val_score(lr_model, X, Y, cv=5)

    return accuracy, cv_scores

def train_svm(X_train, y_train, X_val, y_val):
    # Create an instance of SVM model
    svm_model = SVC(kernel='linear')

    # Train the model
    svm_model.fit(X_train, y_train)

    # Make predictions on the validation set
    y_pred = svm_model.predict(X_val)

    # Calculate the accuracy of the model
    accuracy = accuracy_score(y_val, y_pred)

    # Perform cross-validation
    cv_scores = cross_val_score(svm_model, X, Y, cv=5)

    return accuracy, cv_scores

# Train and evaluate the models
lr_accuracy, lr_cv_scores = train_logistic_regression(X_train, y_train, X_val, y_val)
svm_accuracy, svm_cv_scores = train_svm(X_train, y_train, X_val, y_val)

# Calculate mean and standard deviation of cross-validation scores
lr_mean_cv_score = np.mean(lr_cv_scores)
lr_std_cv_score = np.std(lr_cv_scores)

svm_mean_cv_score = np.mean(svm_cv_scores)
svm_std_cv_score = np.std(svm_cv_scores)

# Model performance summary
model_performance = pd.DataFrame({
    'Model': ['Logistic Regression', 'SVM'],
    'Test Accuracy': [lr_accuracy, svm_accuracy],
    'CV Scores': [lr_cv_scores, svm_cv_scores],
    'Mean CV Score': [lr_mean_cv_score, lr_std_cv_score],
    'Standard Deviation CV Score': [lr_std_cv_score, svm_std_cv_score]
})

# Determine the best model based on mean CV score
best_model_idx = model_performance['Mean CV Score'].idxmax()
best_model = model_performance.loc[best_model_idx]

# Print model performance summary
print("Model Performance:")
print(model_performance)

print("\nBest Model:")
print(f"The best model is {best_model['Model']} with a test set accuracy of {best_model['Test Accuracy']} "
      f"and a mean cross-validation score of {best_model['Mean CV Score']}. "
      f"The model was decided based on the highest accuracy and the highest mean cv score between SVM and LR. The higher these values are, the better the model applies to unseen data")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Model Performance:
                 Model  Test Accuracy  \
0  Logistic Regression       0.788889   
1                  SVM       0.755556   

                                           CV Scores  Mean CV Score  \
0  [0.7611111111111111, 0.7833333333333333, 0.744...       0.778659   
1  [0.7833333333333333, 0.7611111111111111, 0.755...       0.023528   

   Standard Deviation CV Score  
0                     0.023528  
1                     0.022838  

Best Model:
The best model is Logistic Regression with a test set accuracy of 0.7888888888888889 and a mean cross-validation score of 0.7786592178770949. The model was decided based on the highest accuracy and the highest mean cv score between SVM and LR. The higher these values are, the better the model applies to unseen data
